In [1]:
import xpress as xp
import numpy as np
import pandas as pd
import time

In [2]:
print("Loading Data")
# Load distance matrix
distance_matrix = np.loadtxt('distance_matrix_reduce2.csv', delimiter=',')
stations = pd.read_csv('candidate_stations_P300_kmeans_no_snap.csv')
pois = pd.read_csv('reduced_pois2.csv')

Loading Data


In [3]:
I = distance_matrix.shape[0]  # Number of POIs (demand points)
J = distance_matrix.shape[1] # Number of candidate stations

In [4]:
# Parameters
COST_REOPEN_EXISTING = 15000    # £ to reopen existing station (B)
COST_BUILD_NEW = 90000         # £ to build new station (A)
BUDGET = 5000000                # £ total budget (C)

In [5]:
cost_vector = np.where(stations['is_existing_station'], COST_REOPEN_EXISTING, COST_BUILD_NEW)

In [6]:
print("DEFINE MODEL PARAMETERS")
# IMPORTANT: Set the number of facilities to locate
# p = 85  # Change this to test different scenarios

print(f"\n{'='*70}")
print(f"MODEL PARAMETERS")
print(f"{'='*70}")
print(f"Number of POIs (I):                {I}")
print(f"Number of candidate stations (J):  {J}")

DEFINE MODEL PARAMETERS

MODEL PARAMETERS
Number of POIs (I):                885
Number of candidate stations (J):  379


In [7]:
print("CREATE XPRESS PROBLEM")
prob = xp.problem(name='p_centre_reduce')

CREATE XPRESS PROBLEM


/var/folders/tn/v19h6ld131n5p0bkj6b850sr0000gn/T/ipykernel_16380/926094487.py:2: LicenseWarning: Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
  prob = xp.problem(name='p_centre_reduce')


In [ ]:
print("="*70)
print("DEFINE DECISION VARIABLES")
print("="*70)
# Y[j] = 1 if facility j is opened, 0 otherwise
Y = {j: prob.addVariable(vartype=xp.binary, name=f'Y_{j}') for j in range(J)}

# X[i,j] = 1 if demand point i is assigned to station j, 0 otherwise
X = {(i, j): prob.addVariable(vartype=xp.binary, name=f'X_{i}_{j}') 
     for i in range(I) for j in range(J)}

# Q = maximum service distance (minimax objective)
Q = prob.addVariable(name='Q', lb=0)

print(f"Created {J} Y variables (station selection)")
print(f" Created {I*J:,} X variables (POI-station assignments)")
print(f" Created 1 Q variable (minimax distance)")
print(f"  Total variables: {J + I*J + 1:,}")


DEFINE DECISION VARIABLES
Created 379 Y variables (station selection)
 Created 335,415 X variables (POI-station assignments)
 Created 1 W variable (minimax distance)
  Total variables: 335,795


In [9]:
print("="*70)
print("ADD CONSTRAINTS")
print("="*70)
# CONSTRAINT 1: Each POI must be assigned to exactly one station
print("  Adding constraint 1: POI assignment (each POI assigned to exactly one station)...")
for i in range(I):
    prob.addConstraint(xp.Sum(X[i, j] for j in range(J)) == 1)

# CONSTRAINT 2: Exactly p stations must be opened
# print(f"  Adding constraint 2: Facility count (open exactly {p} stations)...")
# prob.addConstraint(xp.Sum(Y[j] for j in range(J)) == p)

# CONSTRAINT 3: Assignment only to open stations (linking constraints)
print("  Adding constraint 3: Linking constraints (assign only to open stations)...")
for i in range(I):
    for j in range(J):
        prob.addConstraint(X[i, j] <= Y[j])

# CONSTRAINT 4: Minimax distance constraint
print("  Adding constraint 4: Minimax distance (Q >= distance to assigned station)...")
for i in range(I):
    prob.addConstraint(
        xp.Sum(distance_matrix[i, j] * X[i, j] for j in range(J)) <= Q)


# CONSTRAINT 5: BUDGET CONSTRAINT (NEW - KEY ADDITION)
print(f"  Constraint 5: Budget constraint (£{BUDGET:,})...")
prob.addConstraint(xp.Sum(cost_vector[j] * Y[j] for j in range(J)) <= BUDGET)

ADD CONSTRAINTS
  Adding constraint 1: POI assignment (each POI assigned to exactly one station)...
  Adding constraint 3: Linking constraints (assign only to open stations)...
  Adding constraint 4: Minimax distance (Q >= distance to assigned station)...
  Constraint 5: Budget constraint (£5,000,000)...


In [10]:
# #CONSTRAINT demand
# for j in range(J):
#         prob.addConstraint(Y[j]<= Z[j] <= C[j] * Y[j])

# for i in range(I):
#     prob.addConstraint(
#         xp.Sum(X[i, j] * demand[i] for j in range(J)) <= Z[j])

In [11]:
print("="*70)
print("SET OBJECTIVE FUNCTION")
print("="*70)
# Minimize Q (the maximum service distance)
prob.setObjective(Q, sense=xp.minimize)

SET OBJECTIVE FUNCTION


In [22]:
print("\n" + "="*70)
print("SOLVING MODEL...")
print("="*70 + "\n")

start_solve = time.time()

# Solve the problem
xp.setOutputEnabled(True)
prob.solve()

solve_time = time.time() - start_solve


SOLVING MODEL...

FICO Xpress v9.7.0, Hyper, solve started 21:22:55, Nov 13, 2025
Heap usage: 475MB (peak 2120MB, 71MB system)
Minimizing MILP p_centre_reduce using up to 8 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
    337186 rows       335795 cols      1342915 elements    335794 entities
Presolved problem has:
    337186 rows       335795 cols      1324352 elements    335794 entities
Presolve finished in 9 seconds
Heap usage: 583MB (peak 2120MB, 71MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 4.60e-05,  9.00e+04] / [ 1.80e-05,  2.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  5.00e+06] / [ 9.25e-02,  5.31e+02]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 8.00e+00,  8.00e+00]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 7.2GB
Starting

In [37]:
    print("\n" + "="*70)
    print("SOLUTION SUMMARY")
    print("="*70)
    

    # Extract optimal minimax distance
    Q_optimal = prob.getSolution(Q)
    
    # Extract which stations to open
    stations_open = []
    for j in range(J):
        if prob.getSolution(Y[j]) > 0.5:  # Binary variable, so > 0.5 means 1
            stations_open.append(j)

    # Classify as existing or new
    existing_open = [j for j in stations_open if cost_vector[j] == 15000]
    new_open = [j for j in stations_open if cost_vector[j] == 90000]
    print(existing_open)
    
    # Calculate total cost
    total_cost = sum(cost_vector[j] for j in stations_open)
    



SOLUTION SUMMARY
[327, 328, 330, 331, 332, 334, 336, 347, 349]


In [24]:
    print(f"\n{'='*70}")
    print(f"OPTIMAL P-CENTRE RESULTS")
    print(f"{'='*70}")
    
    print(f"\n1. STATIONS TO OPEN ({len(stations_open)} out of {J}):")
    print(f"   Station indices: {stations_open}")
    
    print(f"\n   Station details:")
    optimal_stations_list = []
    for idx, j in enumerate(stations_open, 1):
        station_id = stations.iloc[j]['candidate_id']
        lat = station_lat = stations.iloc[j]['centroid_lat']        
        lon = station_lon = stations.iloc[j]['centroid_lon']
        
        optimal_stations_list.append({
            'index': j,
            'rank': idx,
            'station_id': station_id,
            'latitude': lat,
            'longitude': lon
        })
        
        print(f"   {idx:2d}. (Station ID = {station_id})")
    
    print(f"\n2. OPTIMAL MINIMAX DISTANCE:")
    print(f"   Maximum service distance (Q*) = {Q_optimal:.4f} km")
    print(f"   All {I:,} POIs served within {Q_optimal:.4f} km of nearest station")


OPTIMAL P-CENTRE RESULTS

1. STATIONS TO OPEN (63 out of 379):
   Station indices: [2, 3, 5, 6, 7, 9, 11, 13, 14, 17, 19, 20, 21, 22, 24, 28, 35, 40, 47, 61, 67, 70, 73, 84, 92, 103, 113, 119, 137, 146, 151, 155, 157, 165, 167, 168, 170, 171, 174, 175, 176, 185, 188, 204, 210, 221, 233, 252, 255, 259, 277, 278, 287, 294, 327, 328, 330, 331, 332, 334, 336, 347, 349]

   Station details:
    1. (Station ID = 2)
    2. (Station ID = 3)
    3. (Station ID = 5)
    4. (Station ID = 6)
    5. (Station ID = 7)
    6. (Station ID = 9)
    7. (Station ID = 11)
    8. (Station ID = 13)
    9. (Station ID = 14)
   10. (Station ID = 17)
   11. (Station ID = 19)
   12. (Station ID = 20)
   13. (Station ID = 21)
   14. (Station ID = 22)
   15. (Station ID = 24)
   16. (Station ID = 28)
   17. (Station ID = 35)
   18. (Station ID = 40)
   19. (Station ID = 47)
   20. (Station ID = 61)
   21. (Station ID = 67)
   22. (Station ID = 70)
   23. (Station ID = 73)
   24. (Station ID = 84)
   25. (Station 

In [15]:
sol = {(i, j): prob.getSolution(X[i, j]) for i in range(I) for j in range(J)}
solution_matrix = np.zeros((I, J))
for (i, j), val in sol.items():
     solution_matrix[i, j] = val
assignments = np.argmax(solution_matrix, axis=1)

# # Calculate metrics
# total_demand = demand.sum()
# demand_satisfied = sum(demand[assignments == j] for j in stations_open)
    
# selected_capacity = sum(stations.iloc[j]['capacity'] for j in stations_open)
# avg_utilization = 100 * demand_satisfied / selected_capacity if selected_capacity > 0 else 0
all_distances = [distance_matrix[i, assignments[i]] for i in range(I)]
mean_distance = np.mean(all_distances)

In [40]:
    print(f"\n1. OPTIMAL MINIMAX DISTANCE:")
    print(f"   Q* = {Q_optimal:.4f} km")
    
    print(f"\n2. STATIONS OPENED: {len(stations_open)} total")
    print(f"   ├─ Existing (reopen): {len(existing_open)} stations")
    print(f"   └─ New (build): {len(new_open)} stations")
    
    print(f"\n3. COST BREAKDOWN:")
    existing_cost = sum(cost_vector[j] for j in existing_open)
    new_cost = sum(cost_vector[j] for j in new_open)
    print(f"   Reopen existing: {len(existing_open)} × £{COST_REOPEN_EXISTING:,} = £{existing_cost:,}")
    print(f"   Build new:      {len(new_open)} × £{COST_BUILD_NEW:,} = £{new_cost:,}")
    print(f"   Total cost:     £{total_cost:,}")
    print(f"   Budget used:    {100*total_cost/BUDGET:.1f}% of £{BUDGET:,}")
    print(f"   Remaining:      £{BUDGET - total_cost:,}")
    
    
    print(f"\n4. STATIONS TO REOPEN :")
    for j in sorted(existing_open):
        cost = cost_vector[j]
        pois_count = (assignments == j).sum()
        print(f"   [{j:2d}] {str(stations.iloc[j]['candidate_id']):30s} | Cost: £{cost:,} | Serves {pois_count:,} POIs")

    print(f"\n6. STATIONS TO BUILD :")
    for j in sorted(new_open):
        cost = cost_vector[j]
        pois_count = (assignments == j).sum()
        lat, lon = stations.iloc[j]['centroid_lat'], stations.iloc[j]['centroid_lon']
        print(f"   [{j:2d}] Location ({lat:.4f}, {lon:.4f}) | Cost: £{cost:,} | Serves {pois_count:,} POIs")


1. OPTIMAL MINIMAX DISTANCE:
   Q* = 1.4795 km

2. STATIONS OPENED: 63 total
   ├─ Existing (reopen): 9 stations
   └─ New (build): 54 stations

3. COST BREAKDOWN:
   Reopen existing: 9 × £15,000 = £135,000
   Build new:      54 × £90,000 = £4,860,000
   Total cost:     £4,995,000
   Budget used:    99.9% of £5,000,000
   Remaining:      £5,000

4. STATIONS TO REOPEN :
   [327] 327                            | Cost: £15,000 | Serves 12 POIs
   [328] 328                            | Cost: £15,000 | Serves 5 POIs
   [330] 330                            | Cost: £15,000 | Serves 40 POIs
   [331] 331                            | Cost: £15,000 | Serves 9 POIs
   [332] 332                            | Cost: £15,000 | Serves 27 POIs
   [334] 334                            | Cost: £15,000 | Serves 9 POIs
   [336] 336                            | Cost: £15,000 | Serves 26 POIs
   [347] 347                            | Cost: £15,000 | Serves 58 POIs
   [349] 349                            | Cost

In [94]:
import folium
from folium import Circle, PolyLine

# Base map centered on the city
map_center = (pois['lat'].mean(), pois['lon'].mean())
m = folium.Map(location=map_center, zoom_start=13)

for station_idx in stations_open:
    # Station coordinates
    station_lat = stations.iloc[station_idx]['centroid_lat']
    station_lon = stations.iloc[station_idx]['centroid_lon']
    
    # POIs served by this station
    served_pois = [i for i, j in enumerate(assignments) if j == station_idx]
    
    # Calculate maximum coverage radius in kilometers
    if served_pois:
        max_distance_km = max(distance_matrix[i, station_idx] for i in served_pois)
    else:
        max_distance_km = 0.1  # small circle if no POIs
    
    # Draw station marker with Font Awesome bicycle icon
    folium.Marker(
        location=(station_lat, station_lon),
        popup=f"Station {stations.iloc[station_idx]['candidate_id']}",
        icon=folium.Icon(color='darkgreen', icon='bicycle', prefix='fa')
    ).add_to(m)
    
    # Draw transparent coverage circle around station
    Circle(
        location=(station_lat, station_lon),
        radius=max_distance_km * 1000,  # meters
        color='blue',
        fill=True,
        fill_opacity=0.1,
        weight=2,
        popup=f"Coverage area: {max_distance_km:.2f} km"
    ).add_to(m)
    
    # Draw lines and markers for each POI served by this station
    for poi_idx in served_pois:
        poi_lat = pois.iloc[poi_idx]['lat']
        poi_lon = pois.iloc[poi_idx]['lon']
        
        # Draw POI marker as a small red circle
        folium.CircleMarker(
            location=(poi_lat, poi_lon),
            radius=5,
            weight=3,
            color='brown',
            fill=True,
            fill_opacity=100,
            popup=f"POI {pois.iloc[poi_idx]['poi_id']}"
        ).add_to(m)
        
        # Draw line connecting station to POI
        line_coords = [(station_lat, station_lon), (poi_lat, poi_lon)]
        PolyLine(line_coords, color='black', weight=3, opacity=0.7).add_to(m)

# Save map to HTML file
m.save('stations_with_lines_map.html')
print("Map saved as stations_with_lines_map.html")


Map saved as stations_with_lines_map.html
